### The company of this exercise is a social network. They decided to add a feature called: Recommended Friends, i.e. they suggest people you may know.
### A data scientist has built a model to suggest 5 people to each user. These potential friends will be shown on the user newsfeed. At first, the model is tested just on a random subset of users to see how it performs compared to the newsfeed without the new feature.
### The test has been running for some time and your boss asks you to check the results. You are asked to check, for each user, the number of pages visited during their first session since the test started. If this number increased, the test is a success. Specifically, your boss wants to know:
#### (1) Is the test winning? That is, should 100\% of the users see the Recommended Friends feature?
#### (2) Is the test performing similarly for all user segments or are there differences among different segments?
#### (3) If you identified segments that responded differently to the test, can you guess the reason? Would this change your point 1 conclusions?

### Load the package would be used

In [1]:
import pandas as pd
pd.set_option("display.max_columns", 10)
pd.set_option("display.width", 350)

from scipy import stats

import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams.update({"figure.autolayout": True})
import seaborn as sns
sns.set(style = "white")
sns.set(style = "whitegrid", color_codes = True)

In [2]:
user = pd.read_csv("../Datasets/Engagement_Test/user_table.csv")
test = pd.read_csv("../Datasets/Engagement_Test/test_table.csv")

### Look into data

In [ ]:
print(user.shape)
print(test.shape)

print(user.head)
print(test.head)

print(user.info)
print(test.info)

print(len(user["user_id"]) == len(pd.unique(user["user_id"])))
print(len(test["user_id"]) == len(pd.unique(test["user_id"])))

### Data processing

In [ ]:
dat = user.merge(test, on = "user_id", how = "inner")
dat["signup_date"] = pd.to_datetime(dat["signup_date"])
dat["date"] = pd.to_datetime(dat["date"])
dat.head()

#### (1) Is the test winning? That is, should 100% of the users see the Recommended Friends feature?

#### Overall

##### Define the function

In [ ]:
def overall_ttest_mean(dat, variable, test):
    overall = stats.ttest_ind(dat[dat[test] == 1][variable], dat[dat[test] == 0][variable], equal_var = False)
    test_group = dat[dat[test] == 1][variable].mean()
    control_group = dat[dat[test] == 0][variable].mean()
    pvalue = overall.pvalue
    overall_result = {"test_group": test_group, "control_group": control_group, "pvalue": pvalue}
    return overall_result

In [ ]:
overall_ttest_mean(dat = dat, variable = "pages_visited", test = "test")

#### (2) Is the test performing similarly for all user segments or are there differences among different segments?

#### Stratified test

##### Define the function

In [ ]:
def stratified_ttest_mean(dat, stratified, variable, test):
    stratified_result = dat.groupby(stratified)[variable].agg({
        "test_group": lambda x: x[dat[test] == 1].mean(),
        "control_group": lambda x: x[dat[test] == 0].mean(),
        "p_value": lambda x: stats.ttest_ind(x[dat[test] == 1], x[dat[test] == 0], equal_var = False).pvalue
    }).reindex(["test_group", "control_group", "p_value"], axis = 1)
    return stratified_result.sort_values(by = "p_value")

##### Stratified by browser

In [ ]:
stratified_ttest_mean(dat = dat, stratified = "browser", variable = "pages_visited", test = "test").reset_index()

##### Stratified by date

In [ ]:
stratified_ttest_mean(dat = dat, stratified = "date", variable = "pages_visited", test = "test").reset_index().sort_values(by = "date")

#### (3) If you identified segments that responded differently to the test, can you guess the reason? Would this change your point 1 conclusions?

In [ ]:
overall_ttest_mean(dat = dat[dat["browser"] != "Opera"], variable = "pages_visited", test = "test")

Still rush to make the decision to change. Need to consider the novel effect.